In [1]:
include("../src/Julia.jl")

objc[849]: Class TKApplication is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[849]: Class TKMenu is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[849]: Class TKContentView is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[849]: Class TKWindow is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.


parseData (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
gpath = "../../graphs/ipm_grid3_8000_all/";

In [4]:
graphs = readdir(gpath);
len = length(graphs)
graphs = graphs[1:ceil(Int64,len * 0.2)];

In [5]:
for cur in graphs
    path = gpath * cur * "/"
    
    try
        a = readFromFile(path * "graph.mtx");
#         x = readFromFile(path * "x.vec");
        x = zeros(a.n)
        b = readFromFile(path * "b.vec");
        
        tree = []
        for ind in 1:5
            tr = readFromFile(path * "tree$(ind).mtx");
            
            u,v,w = findnz(tr)
            for i in 1:length(w)
                w[i] = a[u[i],v[i]]
            end
            tr = sparse(u,v,w)
            
            push!(tree, tr);
        end
        
        la = lap(a);
        bdef = copy(b)
        bdef = bdef - mean(bdef);
        
        numIts = 100;
        @time for i in 1:5
            @time F = treeSolver(tree[i])
            @time F(bdef);

            # a run on standard Float64 data types
            @time myx64,dbg = pcgV(la, bdef, F, x, maxits=numIts, verbose=false);
            norm(lap(a) * myx64 - bdef) / norm(bdef)
            insert!(dbg, 1, "$(numIts) iterations")
            insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
            writeToFile(path * "_log_julia_tree$(i)_default.txt", dbg)

            # now using higer precision data types
            for hp in [64,128,256,512,1024]
                set_bigfloat_precision(hp)
                laHP = toHighPrecision(la; precision=hp)
                treeHP = toHighPrecision(tree[i]; precision=hp)
                
                xHP = zeros(BigFloat, a.n)
                
                bHP = toHighPrecision(b; precision=hp);
                bHP = bHP - mean(bHP)
                
                FHP = treeSolver(treeHP)

                @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=false);
                insert!(dbg, 1, "$(numIts) iterations")
                insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
                writeToFile(path * "_log_julia_tree$(i)_$(hp).txt", dbg)
            end
        end
            
        println("finished running on ", cur, "\n")
    catch
        println("skipping ", cur, "\n")
    end
end

  0.458369 seconds (639.99 k allocations: 29.183 MB, 1.25% gc time)
  0.152537 seconds (243.80 k allocations: 6.691 MB, 3.91% gc time)
  0.829141 seconds (15.47 M allocations: 383.725 MB, 4.69% gc time)
 50.634441 seconds (261.94 M allocations: 6.627 GB, 36.39% gc time)
 57.985122 seconds (261.76 M allocations: 7.262 GB, 37.86% gc time)
 65.462112 seconds (261.76 M allocations: 8.546 GB, 36.91% gc time)
 75.185997 seconds (261.76 M allocations: 11.116 GB, 35.78% gc time)
 87.766846 seconds (261.76 M allocations: 16.254 GB, 34.77% gc time)
  0.021024 seconds (40.19 k allocations: 4.597 MB)
  0.005726 seconds (151.03 k allocations: 2.794 MB)
  0.797134 seconds (15.26 M allocations: 374.066 MB, 25.85% gc time)
 52.649872 seconds (261.76 M allocations: 6.620 GB, 37.11% gc time)
 56.920468 seconds (261.76 M allocations: 7.262 GB, 37.95% gc time)
 64.506156 seconds (261.76 M allocations: 8.547 GB, 36.82% gc time)
 73.495922 seconds (261.76 M allocations: 11.116 GB, 35.66% gc time)
 87.626228